Add Packages

In [ ]:
#r "nuget:Microsoft.ML, *-*"
#r "nuget:Microsoft.ML.Recommender, *-*"
#r "nuget:Microsoft.Data.Analysis, *-*"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab, *-*"

Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22313.1 Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.22405.1 Microsoft.ML, 2.0.0-preview.22313.1 Microsoft.ML.Recommender, 0.20.0-preview.22313.1

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `Microsoft.DotNet.Interactive.ExtensionLab.dll`

Create strongly-typed dataframes using #!linqify . 
 The #!linqify magic command replaces a Microsoft.Data.Analysis.DataFrame variable with a generated, strongly-typed data frame, allowing the use of LINQ operations over the contained data.

ExplainCode 
 This extension generates Sequence diagrams from csharp code using Mermaid kernel.

Inspect code compilation details using the #!inspect magic command. 
 The #!inspect magic command allows you to see the C# decompilation, IL, and JIT Asm for the code in a C# cell.

Use the #!record magic command to keep a transcript of the code you run. 
 Once you enable transcripts using #!record , each code submission (including re-running cells) is recorded in the specified file. The JSON format used is the same format recognized by the .NET Interactive stdio and http APIs and can be used to replay an interactive session via automation.

Query SQLite databases. 
 This extension adds support for connecting to SQLite databases using the #!connect sqlite magic command. For more information, run a cell using the #!sql magic command.

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;
using Microsoft.ML.Trainers.Recommender;

using System.IO;
using System.Text;

Data Models

In [ ]:
public class SentimentData
{
    [LoadColumn(0)]
    public string SentimentText;

    [LoadColumn(1), ColumnName("Label")]
    public bool Sentiment;
}

public class SentimentPrediction : SentimentData
{
    [ColumnName("Predicted Label")]
    public bool Prediction { get; set; }

    [ColumnName("Probability")]
    public float Probability {get; set; }

    public float Score { get; set; }
}

Load And Split Data

In [ ]:
var mlContext = new MLContext();
var data = mlContext.Data.LoadFromTextFile<SentimentData>("./Data/imdb_labelled.txt", hasHeader: false);

// Split data into testing and training data
var dataSplit = mlContext.Data.TrainTestSplit(data, testFraction: 0.3);

Train The Model

In [ ]:
var pipeline = mlContext.Transforms.Text.FeaturizeText(outputColumnName: "Features", inputColumnName: nameof(SentimentData.SentimentText));
var model = mlContext.BinaryClassification.Trainers.SdcaLogisticRegression(labelColumnName: "Label", featureColumnName: "Features");

var trainingPipeline = pipeline.Append(model);
var trainedModel = trainingPipeline.Fit(dataSplit.TrainSet);

Evaluate The Model

In [ ]:
var eval = trainedModel.Transform(dataSplit.TestSet);
var metrics = mlContext.BinaryClassification.Evaluate(eval);

In [ ]:
metrics

LogLoss,LogLossReduction,Entropy,AreaUnderRocCurve,Accuracy,PositivePrecision,PositiveRecall,NegativePrecision,NegativeRecall,F1Score,AreaUnderPrecisionRecallCurve,ConfusionMatrix
0.8240216438011468,0.17358204203740696,0.9971003604914952,0.8182044515261664,0.7394366197183099,0.7153284671532847,0.7368421052631579,0.7619047619047619,0.7417218543046358,0.7259259259259261,0.818909158207526,"{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 0.7153284671532847, 0.7619047619047619 ], PerClassRecall: [ 0.7368421052631579, 0.7417218543046358 ], Counts: [ [ 98, 35 ], [ 39, 112 ] ], NumberOfClasses: 2 }"


Save The Model

In [ ]:
mlContext.Model.Save(trainedModel, dataSplit.TrainSet.Schema, "./Sentiment.mdl");

Load The Model And Predict

In [ ]:
using (var stream = new FileStream("./Sentiment.mdl", FileMode.Open, FileAccess.Read, FileShare.Read))
{
    var loadedModel = mlContext.Model.Load(stream, out _);
    var predictionEngine = mlContext.Model.CreatePredictionEngine<SentimentData, SentimentPrediction>(loadedModel);

    var newSample = new SentimentData
    {
        SentimentText = "This is good"
    };

    var sentiment = predictionEngine.Predict(newSample);
    Console.WriteLine(sentiment.Prediction);
}

False
